In [3]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F 

import torchvision 
import torchvision.transforms as transforms

The next cell is to get the image from the data set in torchvision 

In [ ]:
train_set=torchvision.datasets.FashionMNIST(
     root='./data/FashionMNIST'
     ,train=True
    ,download=True
     ,transform=transforms.Compose([transforms.ToTensor()])
)

## here is where we are defining our layers 

In [141]:
class CNN(nn.Module):

    def __init__(self):
        super().__init__()
        # convolutional layers to caputre the charachterstic of the picture followed by a fully connected layer 
        self.conv1=nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3,3))
        self.conv2=nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3))
        self.fc1=nn.Linear(in_features=1600, out_features=128)
        self.out=nn.Linear(in_features=128, out_features=10)
    def forward(self, x):
        
        # (1) input layer
        x=x
        print(x.shape)
        # (2) hidden conv layer
        x=self.conv1(x)
        print(x.shape)
        ## the Relu function and max pool 2d (the activation functions)
        ## them directly from the forward function 
        x=F.relu(x)
        x=F.max_pool2d(x, kernel_size=2,stride=2)
        print(x.shape)
        # (3)hidden conv layer
        x=self.conv2(x)
        x=F.relu(x)
        x=F.max_pool2d(x, kernel_size=2,stride=2)
        print(x.shape)
        
        # (4) the hidden Linear layer 
        x=x.reshape(-1,1600)
        print(x.shape)
        x=self.fc1(x)
        x=F.dropout(x,0.5)
        
        # (5) the output layer 
        x=self.out(x)
        return x

In [142]:
torch.set_grad_enabled(False)

**Here we will run our class. the way we defined the nural network using nn.module made it inherit all the charactristic of the nural network from pytorch**



In [143]:
conv_Neural_netwrok=CNN()
print(train_set)

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: ./data/FashionMNIST
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
           )


In [144]:
sample=next(iter(train_set))

In [145]:
image,label=sample
image.shape

torch.Size([1, 28, 28])

# Note

we need a patch with size 1 to be able to pass the image to the network and get prediction 

In [146]:
image.unsqueeze(0).shape

torch.Size([1, 1, 28, 28])

In [147]:
pred=conv_Neural_netwrok(image.unsqueeze(0))      # image size needs to be (patch *in_channels*H*W) 

torch.Size([1, 1, 28, 28])
torch.Size([1, 32, 26, 26])
torch.Size([1, 32, 13, 13])
torch.Size([1, 64, 5, 5])
torch.Size([1, 1600])


In [148]:
pred.shape

torch.Size([1, 10])

## patch some images to pass it to the network and start working with this images. 
so now we use dataloader from pytorch to patch our data. the whole training sample are 6000 samples but for this project and for sake of simplicity we will work with 10 

In [149]:
data_loader = torch.utils.data.DataLoader(
     train_set ,batch_size=10  
)  # the default for batch_size=1  so if you want like 100 samples you need to define it . also our data is 60000 
# so try batching 1000000 will not get you more than the 60000 in our data 

In [150]:
batch=next(iter(data_loader))

In [151]:
images,labels = batch

In [152]:
images.shape

torch.Size([10, 1, 28, 28])

In [153]:
pred=conv_Neural_netwrok(images)

torch.Size([10, 1, 28, 28])
torch.Size([10, 32, 26, 26])
torch.Size([10, 32, 13, 13])
torch.Size([10, 64, 5, 5])
torch.Size([10, 1600])


**This prints in the code is just for the sake of debugging but that can be done using spyder debugging methods with ipdb**

* first we started with a patch of 10 images each one of tem is grey image that is why the color is just one and it has a size of 28 height with 28 width 
* the first convolutional is filtering it using 32 filter each with size 3 . the result of this convolution is 32  image with size (26*26)
* and as a result of the max pool operation and the convolution the size drop more to be 64 filter each with 5 *5 .
* so when it come to make the fullt connected layer we can easier flatten the photo using the flatten input method or we can just change the size again to be 1 filter but combinning this 64 filter with 5 * 5 size into one big tensor again that requires tensorwith size 64*5*5 again that is why we need 1600 

In [154]:
pred.shape

torch.Size([10, 10])

In [155]:
res=pred.argmax(dim=1)

In [156]:
labels

tensor([9, 0, 0, 3, 0, 2, 7, 2, 5, 5])

In [128]:
# to compare labels against the max achieved prediction we can use eq function 
res.eq(labels)

tensor([False, False, False, False, False, False, False, False, False, False])